## Setup / Install

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import defaultdict

In [ ]:
import sys
sys.path.insert(1, '/usr/local/google/home/kmg/')
sys.path.insert(1, '/mnt/c/Users/Kevin Graney/SIG Groupings/')

from sig_groups.rider import Leader, Participant, RiderData, Match
from sig_groups.airtable import LoadLeaders, LoadParticipants, LoadMatches, GetPriorRosters, CreateRoster
from sig_groups.formatting import PrintAvailabilityTable, PrintRosters
from sig_groups.optimizer import AlgorithmTM, Params
from sig_groups.slack import PostRoster, PostRosterStatus
from sig_groups.ride import Roster, Rosters

#### Remove...
import sig_groups
import sig_groups.ride
import sig_groups.rider
import sig_groups.airtable
import sig_groups.formatting
import sig_groups.optimizer
import sig_groups.slack
import importlib
importlib.reload(sig_groups)
importlib.reload(sig_groups.ride)
importlib.reload(sig_groups.rider)
importlib.reload(sig_groups.airtable)
importlib.reload(sig_groups.formatting)
importlib.reload(sig_groups.optimizer)
importlib.reload(sig_groups.slack)

## Parameters
These can be edited if needed.

In [ ]:
#@title Algorithm Parameters
params = Params()

params.start_ride = 5#@param {type:"integer"}
params.finalized_ride = 4#param {type:"integer"}

# Hard limits on group sizes.
params.min_group_size = 4#@param {type:"integer"}
params.max_group_size = 20#@param {type:"integer"}

# The maximum number of groups to assign each week.
params.max_groups = 8#@param {type:"integer"}

# Total number of rides in the program.
params.num_rides = 10#@param {type:"integer"}

# Maximum amount of time to run for, in seconds, per-pass.
params.time_limit = 60#@param {type:"integer"}

# Load Data

In [ ]:
rider_data = RiderData(LoadLeaders(), LoadParticipants(), LoadMatches())

In [ ]:
PrintAvailabilityTable(rider_data.AllLeaders(), params.num_rides)

In [ ]:
prior_rosters = GetPriorRosters(rider_data)
PrintRosters(prior_rosters)

# Generate Rosters

In [ ]:
from IPython.display import HTML, Javascript, display

display(HTML("""<div id="root"></div>"""))

display(Javascript(url="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"))

display(Javascript('''
(async () => {
  google.colab.kernel.comms.registerTarget('rosters', (comm, message) => {
    $("#root").empty();
    var text = '';
    for (ride in message.data) {
      var ride_div = $(`<div id="Ride ${ride}" style="margin-top: 20pt;"><strong>Ride ${ride}</strong><br /></div>`);
      for (group in message.data[ride]) {
        var group_div = $(`<span id="Group ${group}" style="font-size: 75%; vertical-align: top; display: inline-block; margin-right: 5pt; width: 30ex; border: 1px solid red;"></span>`);
        ride_div.append(group_div);
        var ul = group_div.append(`<ul></ul>`)
        for (participant of message.data[ride][group]) {
          var li = $(`<li>${participant}</li>`)
          ul.append(li)
        }
      }
      $("#root").append(ride_div);
    }
  });
})()'''))

In [ ]:
alg = AlgorithmTM(rider_data, prior_rosters, params)
rosters = alg.Solve()

In [ ]:
PrintRosters(prior_rosters)

In [ ]:
rides_together = defaultdict(lambda: 0)
match_hist = defaultdict(lambda: 0)
for r in rosters:
  for p1 in r.riders:
    for p2 in r.riders:
      rides_together[(p1.id, p2.id)] += 1
      match_hist[rider_data.GetMatchScore(p1.id, p2.id)] += 1

people = rider_data.AllFtRiders()
seen = set()
common_pairings = []
hist = defaultdict(lambda: 0)
for p1 in people:
  for p2 in people:
    if (p1 != p2) and ((p2, p1) not in seen):
      hist[rides_together[(p1.id, p2.id)]] += 1
      common_pairings.append((rides_together[(p1.id, p2.id)], p1.name, p2.name))
      seen.add((p1, p2))

plt.bar(hist.keys(), hist.values())

People that never get to ride together.

In [ ]:
for (_, p1, p2) in filter(lambda x: x[0] == 0, common_pairings):
  print("{: <30} {: >30}".format(p1, p2))

People who ride together the most.

In [ ]:
for (n, p1, p2) in sorted(common_pairings, reverse=True)[0:50]:
  print("{: <30} {: 2} {: >30}".format(p1, n, p2))

# Upload to Airtable

In [ ]:
for r in rosters:
  if r.ride == 0:
    print(r )

In [ ]:
for r in rosters:
  CreateRoster(r)

In [ ]:
m = defaultdict(lambda: [])
for r in rider_data.AllParticipants():
  if r.Invalid():
    continue
  m[r.mentor].append(r)

for (k,v) in m.items():
  print(rider_data.Rider(k).name, [x.name for x in v])

In [ ]:
rides_together = defaultdict(lambda: 0)
match_hist = defaultdict(lambda: 0)
for r in [x for x in rosters if x.ride < 2]:
  for p1 in r.riders:
    for p2 in r.riders:
      rides_together[(p1.id, p2.id)] += 1
      match_hist[RIDERS.GetMatchScore(p1.id, p2.id)] += 1

people = RIDERS.AllFtRiders()
seen = set()
common_pairings = []
hist = defaultdict(lambda: 0)
for p1 in people:
  for p2 in people:
    if (p1 != p2) and ((p2, p1) not in seen):
      hist[rides_together[(p1.id, p2.id)]] += 1
      common_pairings.append((rides_together[(p1.id, p2.id)], p1.name, p2.name))
      seen.add((p1, p2))

plt.bar(hist.keys(), hist.values())
for (n, p1, p2) in sorted(common_pairings, reverse=True)[0:50]:
  print("{: <30} {: 2} {: >30}".format(p1, n, p2))

In [ ]:
def MentorPair(p1, p2):
  if p1.IsLeader() == p2.IsLeader():
    return False
  if not p1.IsLeader():
    return p1.mentor == p2.id
  if not p2.IsLeader():
    return p2.mentor == p1.id

for p1 in people:
  for p2 in people:
    if MentorPair(p1, p2):
      print(p1.name, p2.name)

# Update Slack rosters

In [ ]:
rider_data = RiderData(LoadLeaders(), LoadParticipants(), LoadMatches())
all_rosters = GetPriorRosters(rider_data)

In [ ]:
PrintRosters(all_rosters)

In [ ]:
rides_together = defaultdict(lambda: 0)
rides_together_so_far = defaultdict(lambda: 0)
match_hist = defaultdict(lambda: 0)
        
for r in all_rosters:
  for p1 in r.riders:
    for p2 in r.riders:
      rides_together[(p1.id, p2.id)] += 1
      if r.ride <= params.start_ride:
        rides_together_so_far[(p1.id, p2.id)] += 1
      match_hist[rider_data.GetMatchScore(p1.id, p2.id)] += 1

people = rider_data.AllRiders()
seen = set()
common_pairings = []
hist = defaultdict(lambda: 0)
for p1 in people:
  for p2 in people:
    if (p1 != p2) and ((p2, p1) not in seen):
      hist[rides_together[(p1.id, p2.id)]] += 1
      common_pairings.append((rides_together[(p1.id, p2.id)], p1.name, p2.name))
      seen.add((p1, p2))

plt.bar(hist.keys(), hist.values())

In [ ]:
final_pair_matrix = np.zeros(shape=(len(people), len(people)))
modeled_pair_matrix = np.zeros(shape=(len(people), len(people)))
for i, p1 in enumerate(people):
  for j, p2 in enumerate(people):
    final_pair_matrix[i][j] = rides_together_so_far[(p1.id, p2.id)]
    modeled_pair_matrix[i][j] = rides_together[(p1.id, p2.id)]

labels = [p.name for p in people]
       
maxval = np.max([np.max(final_pair_matrix), np.max(modeled_pair_matrix)])        
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,20))
ax1.set_title('Up to this Ride (Ride %d)' % (params.start_ride + 1))
ax1.set_xticks(np.arange(0, len(people), 1.0), labels=labels, minor=True, rotation=90, fontsize='x-small')
ax1.set_xticks(np.arange(0, len(people), 1.0), labels=labels, minor=False, rotation=90, fontsize='x-small')
ax1.set_yticks(np.arange(0, len(people), 1.0), labels=labels, minor=True, fontsize='x-small')
ax1.set_yticks(np.arange(0, len(people), 1.0), labels=labels, minor=False, fontsize='x-small')
ax2.set_xticks(np.arange(0, len(people), 1.0), labels=labels, minor=True, rotation=90, fontsize='x-small')
ax2.set_xticks(np.arange(0, len(people), 1.0), labels=labels, minor=False, rotation=90, fontsize='x-small')
for xtick, person in zip(ax1.get_xticklabels(), people):
    if not person.IsLeader():
        xtick.set_color('r')

#ax1.set_xticks(range(0, len(people)), labels=labels, minor=False, rotation=90, fontsize='xx-small')

#ax1.set_xticklabels(labels, rotation=90, horizontalalignment='right')
ax1.matshow(final_pair_matrix, vmin=0, vmax=maxval, cmap=plt.cm.viridis)
ax2.set_title('Modeled to Finish')
ax2.matshow(modeled_pair_matrix, vmin=0, vmax=maxval, cmap=plt.cm.viridis)
#fig.colorbar(plot2)
fig.savefig('/tmp/pairings.png', facecolor='white', transparent=False, bbox_inches='tight')

In [ ]:
PostRosterStatus()

In [ ]:
all_rosters = GetPriorRosters(rider_data)
for ride in range(params.start_ride, params.num_rides):
  rosters = Rosters(r for r in all_rosters if r.ride == ride)
  PostRoster(rosters)